In [1]:
import os
import tensorflow as tf
from tensorflow import keras as keras
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder
import numpy as np
import pandas as pd
from matplotlib import pyplot as plt
#os.environ['CUDA_VISIBLE_DEVICES'] = '-1'

if tf.test.gpu_device_name():
    print('GPU found')
else:
    print("No GPU found")

GPU found


Data Preparing

In [2]:
data_dir_path = "../data"


train_df = pd.read_csv(f'{data_dir_path}/emnist-byclass-train.csv', header=None)
test_df = pd.read_csv(f"{data_dir_path}/emnist-byclass-test.csv", header=None)

In [3]:
def convert_data_csv_to_numpy(data, sorting = False): # pandas 통해 읽은 csv data numpy 형태로 변경
  if sorting == True:
    data = data.sort_values(by=[0], axis=0)

  label = np.array(data[0]) # csv file 에서 0번째 colum은 index임
  only_data = np.array(data.drop([0], axis = 1)) # csv file에서 0번 째 colum 탈락 -> data만 남게 됨
  # only_data = tf.convert_to_tensor(only_data, dtype=tf.float32)
  only_data = tf.keras.utils.normalize(only_data, axis=-1, order=2).reshape((-1, 28, 28, 1))
  return only_data, label

In [4]:
X_train, y_train  = convert_data_csv_to_numpy(train_df)
X_test, y_test  = convert_data_csv_to_numpy(test_df)

train_df = None
test_df = None

In [5]:

# X_train = np.repeat(X_train, 3, axis=-1)
# X_test = np.repeat(X_test, 3, axis=-1)

X_train, X_val, y_train, y_val = train_test_split(X_train, y_train, test_size=0.2, random_state=42)


# X_train = tf.image.resize(X_train, [32,32])
# X_val = tf.image.resize(X_val, [32,32])
# X_test = tf.image.resize(X_test, [32,32])

In [6]:
print(X_train.shape,X_val.shape, X_test.shape)

(558345, 28, 28, 1) (139587, 28, 28, 1) (116323, 28, 28, 1)


In [7]:
y_train = keras.utils.to_categorical(y_train)
y_val = keras.utils.to_categorical(y_val)
y_test = keras.utils.to_categorical(y_test)

print(y_train.shape,y_val.shape, y_test.shape)

(558345, 62) (139587, 62) (116323, 62)


In [8]:
class ResnetBlock(keras.models.Model):
    """
    A standard resnet block.
    """

    def __init__(self, channels: int, down_sample=False):
        """
        channels: same as number of convolution kernels
        """
        super().__init__()

        self.__channels = channels
        self.__down_sample = down_sample
        self.__strides = [2, 1] if down_sample else [1, 1]

        KERNEL_SIZE = (3, 3)
        # use He initialization, instead of Xavier (a.k.a 'glorot_uniform' in Keras), as suggested in [2]
        INIT_SCHEME = "he_normal"

        self.conv_1 = keras.layers.Conv2D(self.__channels, strides=self.__strides[0],
                             kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME)
        self.bn_1 = keras.layers.BatchNormalization()
        self.conv_2 = keras.layers.Conv2D(self.__channels, strides=self.__strides[1],
                             kernel_size=KERNEL_SIZE, padding="same", kernel_initializer=INIT_SCHEME)
        self.bn_2 = keras.layers.BatchNormalization()
        self.merge = keras.layers.Add()

        if self.__down_sample:
            # perform down sampling using stride of 2, according to [1].
            self.res_conv = keras.layers.Conv2D(
                self.__channels, strides=2, kernel_size=(1, 1), kernel_initializer=INIT_SCHEME, padding="same")
            self.res_bn = keras.layers.BatchNormalization()

    def call(self, inputs):
        res = inputs

        x = self.conv_1(inputs)
        x = self.bn_1(x)
        x = tf.nn.relu(x)
        x = self.conv_2(x)
        x = self.bn_2(x)

        if self.__down_sample:
            res = self.res_conv(res)
            res = self.res_bn(res)

        # if not perform down sample, then add a shortcut directly
        x = self.merge([x, res])
        out = tf.nn.relu(x)
        return out


# class ResNet18(keras.models.Model):

#     def __init__(self, num_classes, **kwargs):
#         """
#             num_classes: number of classes in specific classification task.
#         """
#         super().__init__(**kwargs)
#         self.conv_1 = keras.layers.Conv2D(64, (7, 7), strides=2,
#                              padding="same", kernel_initializer="he_normal")
#         self.init_bn = keras.layers.BatchNormalization()
#         self.pool_2 = keras.layers.MaxPool2D(pool_size=(2, 2), strides=2, padding="same")
#         self.res_1_1 = ResnetBlock(64)
#         self.res_1_2 = ResnetBlock(64)
#         self.res_2_1 = ResnetBlock(128, down_sample=True)
#         self.res_2_2 = ResnetBlock(128)
#         self.res_3_1 = ResnetBlock(256, down_sample=True)
#         self.res_3_2 = ResnetBlock(256)
#         self.res_4_1 = ResnetBlock(512, down_sample=True)
#         self.res_4_2 = ResnetBlock(512)
#         self.avg_pool = keras.layers.GlobalAveragePooling2D()
#         self.flat = keras.layers.Flatten()
#         self.fc = keras.layers.Dense(num_classes, activation="softmax")

#     def call(self, inputs):
#         out = self.conv_1(inputs)
#         out = self.init_bn(out)
#         out = tf.nn.relu(out)
#         out = self.pool_2(out)
#         for res_block in [self.res_1_1, self.res_1_2, self.res_2_1, self.res_2_2, self.res_3_1, self.res_3_2, self.res_4_1, self.res_4_2]:
#             out = res_block(out)
#         out = self.avg_pool(out)
#         out = self.flat(out)
#         out = self.fc(out)
#         return out


# resnet_model = ResNet18(62)
# resnet_model.build(input_shape = (None,28,28,1))
# #use categorical_crossentropy since the label is one-hot encoded

# # from keras.optimizers import SGD
# # opt = SGD(learning_rate=0.1,momentum=0.9,decay = 1e-04) #parameters suggested by He [1]
# resnet_model.compile(optimizer = "adam",loss='categorical_crossentropy', metrics=["accuracy"]) 
# resnet_model.summary()

In [ ]:
class ResBaseModel(keras.models.Model):

    def __init__(self, num_classes, **kwargs):
        """
            num_classes: number of classes in specific classification task.
        """
        super().__init__(**kwargs)
        self.conv_1 = keras.layers.Conv2D(32, (7, 7), strides=2,
                             padding="same", kernel_initializer="he_normal")
        self.init_bn = keras.layers.BatchNormalization()
        self.pool_2 = keras.layers.MaxPool2D(pool_size=(2, 2), strides=2, padding="same")
        self.res_1_1 = ResnetBlock(32)
        self.res_1_2 = ResnetBlock(32)
        self.res_2_1 = ResnetBlock(64, down_sample=True)
        self.res_2_2 = ResnetBlock(64)
        self.res_3_1 = ResnetBlock(128, down_sample=True)
        self.res_3_2 = ResnetBlock(128)
        
        self.res_num_1 = ResnetBlock(256, down_sample=True)
        self.res_num_2 = ResnetBlock(256)

        self.res_upper_1 = ResnetBlock(256, down_sample=True)
        self.res_upper_2 = ResnetBlock(256)        
        self.res_upper_3 = ResnetBlock(256)        


        self.res_lower_1 = ResnetBlock(256, down_sample=True)
        self.res_lower_2 = ResnetBlock(256)
        self.res_lower_3 = ResnetBlock(256)

        self.num_avg_pool = keras.layers.GlobalAveragePooling2D()
        self.upper_avg_pool = keras.layers.GlobalAveragePooling2D()
        self.lower_avg_pool = keras.layers.GlobalAveragePooling2D()

        self.num_flat = keras.layers.Flatten()
        self.upper_flat = keras.layers.Flatten()
        self.lower_flat = keras.layers.Flatten()

        self.fc_out = keras.layers.Dense(num_classes, activation="softmax")

    def call(self, inputs):
        out = self.conv_1(inputs)
        out = self.init_bn(out)
        out = tf.nn.relu(out)
        out = self.pool_2(out)
        for res_block in [self.res_1_1, self.res_1_2, self.res_2_1, self.res_2_2, self.res_3_1, self.res_3_2]:
            out = res_block(out)
        
        num_out = self.res_num_1(out)
        num_out = self.res_num_2(num_out)
        num_out = self.num_avg_pool(num_out)
        num_out = self.num_flat(num_out)

        upper_out = self.res_upper_1(out)
        upper_out = self.res_upper_2(upper_out)
        upper_out = self.res_upper_3(upper_out)
        upper_out = self.upper_avg_pool(upper_out)
        upper_out = self.upper_flat(upper_out)

        lower_out = self.res_lower_1(out)
        lower_out = self.res_lower_2(lower_out)
        lower_out = self.res_lower_3(lower_out)
        lower_out = self.lower_avg_pool(lower_out)
        lower_out = self.lower_flat(lower_out)

        out = keras.layers.Concatenate()([num_out, upper_out, lower_out])

        out = self.fc_out(out)
        return out


resnet_model = ResBaseModel(62)
resnet_model.build(input_shape = (None,28,28,1))
#use categorical_crossentropy since the label is one-hot encoded

# from keras.optimizers import SGD
# opt = SGD(learning_rate=0.1,momentum=0.9,decay = 1e-04) #parameters suggested by He [1]
resnet_model.compile(optimizer = "adam",loss='categorical_crossentropy', metrics=["accuracy"]) 
resnet_model.summary()

In [ ]:
class Res_3x3(keras.models.Model):

    def __init__(self, num_classes, **kwargs):
        """
            num_classes: number of classes in specific classification task.
        """
        super().__init__(**kwargs)
        self.conv_1 = keras.layers.Conv2D(16, (3, 3), strides=2,
                             padding="same", kernel_initializer="he_normal")
        self.init_bn = keras.layers.BatchNormalization()
        self.pool_2 = keras.layers.MaxPool2D(pool_size=(2, 2), strides=2, padding="same")
        self.res_1_1 = ResnetBlock(16)
        self.res_1_2 = ResnetBlock(16)
        self.res_2_1 = ResnetBlock(32, down_sample=True)
        self.res_2_2 = ResnetBlock(32)
        self.res_3_1 = ResnetBlock(64, down_sample=True)
        self.res_3_2 = ResnetBlock(64)
        
        self.res_num_1 = ResnetBlock(128, down_sample=True)
        self.res_num_2 = ResnetBlock(128)

        self.res_upper_1 = ResnetBlock(128, down_sample=True)
        self.res_upper_2 = ResnetBlock(128)        
        self.res_upper_3 = ResnetBlock(128)        


        self.res_lower_1 = ResnetBlock(128, down_sample=True)
        self.res_lower_2 = ResnetBlock(128)
        self.res_lower_3 = ResnetBlock(128)

        self.num_avg_pool = keras.layers.GlobalAveragePooling2D()
        self.upper_avg_pool = keras.layers.GlobalAveragePooling2D()
        self.lower_avg_pool = keras.layers.GlobalAveragePooling2D()

        self.num_flat = keras.layers.Flatten()
        self.upper_flat = keras.layers.Flatten()
        self.lower_flat = keras.layers.Flatten()

        self.fc_out = keras.layers.Dense(num_classes, activation="softmax")

    def call(self, inputs):
        out = self.conv_1(inputs)
        out = self.init_bn(out)
        out = tf.nn.relu(out)
        out = self.pool_2(out)
        for res_block in [self.res_1_1, self.res_1_2, self.res_2_1, self.res_2_2, self.res_3_1, self.res_3_2]:
            out = res_block(out)
        
        num_out = self.res_num_1(out)
        num_out = self.res_num_2(num_out)
        num_out = self.num_avg_pool(num_out)
        num_out = self.num_flat(num_out)

        upper_out = self.res_upper_1(out)
        upper_out = self.res_upper_2(upper_out)
        upper_out = self.res_upper_3(upper_out)
        upper_out = self.upper_avg_pool(upper_out)
        upper_out = self.upper_flat(upper_out)

        lower_out = self.res_lower_1(out)
        lower_out = self.res_lower_2(lower_out)
        lower_out = self.res_lower_3(lower_out)
        lower_out = self.lower_avg_pool(lower_out)
        lower_out = self.lower_flat(lower_out)

        out = keras.layers.Concatenate()([num_out, upper_out, lower_out])

        out = self.fc_out(out)
        return out


In [ ]:
class Res_5x5(keras.models.Model):

    def __init__(self, num_classes, **kwargs):
        """
            num_classes: number of classes in specific classification task.
        """
        super().__init__(**kwargs)
        self.conv_1 = keras.layers.Conv2D(16, (5, 5), strides=2,
                             padding="same", kernel_initializer="he_normal")
        self.init_bn = keras.layers.BatchNormalization()
        self.pool_2 = keras.layers.MaxPool2D(pool_size=(2, 2), strides=2, padding="same")
        self.res_1_1 = ResnetBlock(16)
        self.res_1_2 = ResnetBlock(16)
        self.res_2_1 = ResnetBlock(32, down_sample=True)
        self.res_2_2 = ResnetBlock(32)
        self.res_3_1 = ResnetBlock(64, down_sample=True)
        self.res_3_2 = ResnetBlock(64)
        
        self.res_num_1 = ResnetBlock(128, down_sample=True)
        self.res_num_2 = ResnetBlock(128)

        self.res_upper_1 = ResnetBlock(128, down_sample=True)
        self.res_upper_2 = ResnetBlock(128)        
        self.res_upper_3 = ResnetBlock(128)        


        self.res_lower_1 = ResnetBlock(128, down_sample=True)
        self.res_lower_2 = ResnetBlock(128)
        self.res_lower_3 = ResnetBlock(128)

        self.num_avg_pool = keras.layers.GlobalAveragePooling2D()
        self.upper_avg_pool = keras.layers.GlobalAveragePooling2D()
        self.lower_avg_pool = keras.layers.GlobalAveragePooling2D()

        self.num_flat = keras.layers.Flatten()
        self.upper_flat = keras.layers.Flatten()
        self.lower_flat = keras.layers.Flatten()

        self.fc_out = keras.layers.Dense(num_classes, activation="softmax")

    def call(self, inputs):
        out = self.conv_1(inputs)
        out = self.init_bn(out)
        out = tf.nn.relu(out)
        out = self.pool_2(out)
        for res_block in [self.res_1_1, self.res_1_2, self.res_2_1, self.res_2_2, self.res_3_1, self.res_3_2]:
            out = res_block(out)
        
        num_out = self.res_num_1(out)
        num_out = self.res_num_2(num_out)
        num_out = self.num_avg_pool(num_out)
        num_out = self.num_flat(num_out)

        upper_out = self.res_upper_1(out)
        upper_out = self.res_upper_2(upper_out)
        upper_out = self.res_upper_3(upper_out)
        upper_out = self.upper_avg_pool(upper_out)
        upper_out = self.upper_flat(upper_out)

        lower_out = self.res_lower_1(out)
        lower_out = self.res_lower_2(lower_out)
        lower_out = self.res_lower_3(lower_out)
        lower_out = self.lower_avg_pool(lower_out)
        lower_out = self.lower_flat(lower_out)

        out = keras.layers.Concatenate()([num_out, upper_out, lower_out])

        out = self.fc_out(out)
        return out


In [ ]:
class Res_5x5_v2(keras.models.Model):

    def __init__(self, num_classes, **kwargs):
        """
            num_classes: number of classes in specific classification task.
        """
        super().__init__(**kwargs)
        self.conv_1 = keras.layers.Conv2D(32, (5, 5), strides=2,
                             padding="same", kernel_initializer="he_normal")
        self.init_bn = keras.layers.BatchNormalization()
        self.pool_2 = keras.layers.MaxPool2D(pool_size=(2, 2), strides=2, padding="same")
        self.res_1_1 = ResnetBlock(32)
        self.res_1_2 = ResnetBlock(32)
        self.res_2_1 = ResnetBlock(64, down_sample=True)
        self.res_2_2 = ResnetBlock(64)
        self.res_3_1 = ResnetBlock(128, down_sample=True)
        self.res_3_2 = ResnetBlock(128)
        
        self.res_num_1 = ResnetBlock(256, down_sample=True)
        self.res_num_2 = ResnetBlock(256)

        self.res_upper_1 = ResnetBlock(256, down_sample=True)
        self.res_upper_2 = ResnetBlock(256)        
        self.res_upper_3 = ResnetBlock(256)        


        self.res_lower_1 = ResnetBlock(256, down_sample=True)
        self.res_lower_2 = ResnetBlock(256)
        self.res_lower_3 = ResnetBlock(256)

        self.num_avg_pool = keras.layers.GlobalAveragePooling2D()
        self.upper_avg_pool = keras.layers.GlobalAveragePooling2D()
        self.lower_avg_pool = keras.layers.GlobalAveragePooling2D()

        self.num_flat = keras.layers.Flatten()
        self.upper_flat = keras.layers.Flatten()
        self.lower_flat = keras.layers.Flatten()

        self.fc_out = keras.layers.Dense(num_classes, activation="softmax")

    def call(self, inputs):
        out = self.conv_1(inputs)
        out = self.init_bn(out)
        out = tf.nn.relu(out)
        out = self.pool_2(out)
        for res_block in [self.res_1_1, self.res_1_2, self.res_2_1, self.res_2_2, self.res_3_1, self.res_3_2]:
            out = res_block(out)
        
        num_out = self.res_num_1(out)
        num_out = self.res_num_2(num_out)
        num_out = self.num_avg_pool(num_out)
        num_out = self.num_flat(num_out)

        upper_out = self.res_upper_1(out)
        upper_out = self.res_upper_2(upper_out)
        upper_out = self.res_upper_3(upper_out)
        upper_out = self.upper_avg_pool(upper_out)
        upper_out = self.upper_flat(upper_out)

        lower_out = self.res_lower_1(out)
        lower_out = self.res_lower_2(lower_out)
        lower_out = self.res_lower_3(lower_out)
        lower_out = self.lower_avg_pool(lower_out)
        lower_out = self.lower_flat(lower_out)

        out = keras.layers.Concatenate()([num_out, upper_out, lower_out])

        out = self.fc_out(out)
        return out



In [9]:
class Res_5x5x_v3(keras.models.Model):

    def __init__(self, num_classes, **kwargs):
        """
            num_classes: number of classes in specific classification task.
        """
        super().__init__(**kwargs)
        self.conv_1 = keras.layers.Conv2D(32, (5, 5), strides=1,
                             padding="same", kernel_initializer="he_normal")
        self.init_bn = keras.layers.BatchNormalization()
        self.pool_2 = keras.layers.MaxPool2D(pool_size=(2, 2), strides=2, padding="same")
        self.res_1_1 = ResnetBlock(32)
        self.res_1_2 = ResnetBlock(32)
        self.res_2_1 = ResnetBlock(64, down_sample=True)
        self.res_2_2 = ResnetBlock(64)
        self.res_3_1 = ResnetBlock(128, down_sample=True)
        self.res_3_2 = ResnetBlock(128)
        
        self.res_num_1 = ResnetBlock(256, down_sample=True)
        self.res_num_2 = ResnetBlock(256)

        self.res_upper_1 = ResnetBlock(256, down_sample=True)
        self.res_upper_2 = ResnetBlock(256)        
        self.res_upper_3 = ResnetBlock(256)        


        self.res_lower_1 = ResnetBlock(256, down_sample=True)
        self.res_lower_2 = ResnetBlock(256)
        self.res_lower_3 = ResnetBlock(256)

        self.num_avg_pool = keras.layers.GlobalAveragePooling2D()
        self.upper_avg_pool = keras.layers.GlobalAveragePooling2D()
        self.lower_avg_pool = keras.layers.GlobalAveragePooling2D()

        self.num_flat = keras.layers.Flatten()
        self.upper_flat = keras.layers.Flatten()
        self.lower_flat = keras.layers.Flatten()

        self.fc_out = keras.layers.Dense(num_classes, activation="softmax")

    def call(self, inputs):
        out = self.conv_1(inputs)
        out = self.init_bn(out)
        out = tf.nn.relu(out)
        out = self.pool_2(out)
        for res_block in [self.res_1_1, self.res_1_2, self.res_2_1, self.res_2_2, self.res_3_1, self.res_3_2]:
            out = res_block(out)
        
        num_out = self.res_num_1(out)
        num_out = self.res_num_2(num_out)
        num_out = self.num_avg_pool(num_out)
        num_out = self.num_flat(num_out)

        upper_out = self.res_upper_1(out)
        upper_out = self.res_upper_2(upper_out)
        upper_out = self.res_upper_3(upper_out)
        upper_out = self.upper_avg_pool(upper_out)
        upper_out = self.upper_flat(upper_out)

        lower_out = self.res_lower_1(out)
        lower_out = self.res_lower_2(lower_out)
        lower_out = self.res_lower_3(lower_out)
        lower_out = self.lower_avg_pool(lower_out)
        lower_out = self.lower_flat(lower_out)

        out = keras.layers.Concatenate()([num_out, upper_out, lower_out])

        out = self.fc_out(out)
        return out



In [25]:
class Res_5x5_v4(keras.models.Model):

    def __init__(self, num_classes, **kwargs):
        """
            num_classes: number of classes in specific classification task.
        """
        super().__init__(**kwargs)
        input_ch = 32
        self.conv_1 = keras.layers.Conv2D(input_ch, (5, 5), strides=1,
                             padding="same", kernel_initializer="he_normal")
        self.init_bn = keras.layers.BatchNormalization()
        self.pool_2 = keras.layers.MaxPool2D(pool_size=(2, 2), strides=1, padding="same")
        self.res_1_1 = ResnetBlock(input_ch)
        self.res_1_2 = ResnetBlock(input_ch)
        self.res_2_1 = ResnetBlock(input_ch*2, down_sample=True)
        self.res_2_2 = ResnetBlock(input_ch*2)
        self.res_3_1 = ResnetBlock(input_ch*4, down_sample=True)
        self.res_3_2 = ResnetBlock(input_ch*4)
        
        self.res_num_3 = ResnetBlock(input_ch*8, down_sample=True)
        self.res_num_4 = ResnetBlock(input_ch*8)
        
        self.res_upper_1 = ResnetBlock(input_ch*8, down_sample=True)
        self.res_upper_2 = ResnetBlock(input_ch*8)
        self.res_upper_3 = ResnetBlock(input_ch*8)
   

        self.res_lower_1 = ResnetBlock(input_ch*8, down_sample=True)
        self.res_lower_2 = ResnetBlock(input_ch*8)
        self.res_lower_3 = ResnetBlock(input_ch*8)

        self.num_avg_pool = keras.layers.GlobalAveragePooling2D()
        self.upper_avg_pool = keras.layers.GlobalAveragePooling2D()
        self.lower_avg_pool = keras.layers.GlobalAveragePooling2D()

        self.num_flat = keras.layers.Flatten()
        self.upper_flat = keras.layers.Flatten()
        self.lower_flat = keras.layers.Flatten()

        self.fc_out = keras.layers.Dense(num_classes, activation="softmax")

    def call(self, inputs):
        out = self.conv_1(inputs)
        out = self.init_bn(out)
        out = tf.nn.relu(out)
        out = self.pool_2(out)
        for res_block in [self.res_1_1, self.res_1_2, self.res_2_1, self.res_2_2, self.res_3_1, self.res_3_2]:
            out = res_block(out)
        
        num_out = self.res_num_1(out)
        num_out = self.res_num_2(num_out)
        num_out = self.num_avg_pool(num_out)
        num_out = self.num_flat(num_out)

        upper_out = self.res_upper_1(out)
        upper_out = self.res_upper_2(upper_out)
        upper_out = self.res_upper_3(upper_out)
        upper_out = self.upper_avg_pool(upper_out)
        upper_out = self.upper_flat(upper_out)

        lower_out = self.res_lower_1(out)
        lower_out = self.res_lower_2(lower_out)
        lower_out = self.res_lower_3(lower_out)
        lower_out = self.lower_avg_pool(lower_out)
        lower_out = self.lower_flat(lower_out)

        out = keras.layers.Concatenate()([num_out, upper_out, lower_out])

        out = self.fc_out(out)
        return out



In [25]:
class Res_5x5_v5(keras.models.Model):

    def __init__(self, num_classes, **kwargs):
        """
            num_classes: number of classes in specific classification task.
        """
        super().__init__(**kwargs)
        input_ch = 32
        self.conv_1 = keras.layers.Conv2D(input_ch, (5, 5), strides=2,
                             padding="same", kernel_initializer="he_normal")
        self.init_bn = keras.layers.BatchNormalization()
        self.pool_2 = keras.layers.MaxPool2D(pool_size=(2, 2), strides=2, padding="same")
        self.res_1_1 = ResnetBlock(input_ch)
        self.res_1_2 = ResnetBlock(input_ch)
        self.res_2_1 = ResnetBlock(input_ch*2, down_sample=True)
        self.res_2_2 = ResnetBlock(input_ch*2)
        self.res_3_1 = ResnetBlock(input_ch*4, down_sample=True)
        self.res_3_2 = ResnetBlock(input_ch*4)
        
        self.res_num_3 = ResnetBlock(input_ch*8, down_sample=True)
        self.res_num_4 = ResnetBlock(input_ch*8)
        
        self.res_upper_1 = ResnetBlock(input_ch*8, down_sample=True)
        self.res_upper_2 = ResnetBlock(input_ch*8)
        self.res_upper_3 = ResnetBlock(input_ch*8)
   

        self.res_lower_1 = ResnetBlock(input_ch*8, down_sample=True)
        self.res_lower_2 = ResnetBlock(input_ch*8)
        self.res_lower_3 = ResnetBlock(input_ch*8)

        self.num_avg_pool = keras.layers.GlobalAveragePooling2D()
        self.upper_avg_pool = keras.layers.GlobalAveragePooling2D()
        self.lower_avg_pool = keras.layers.GlobalAveragePooling2D()

        self.num_flat = keras.layers.Flatten()
        self.upper_flat = keras.layers.Flatten()
        self.lower_flat = keras.layers.Flatten()

        self.fc_num = keras.layers.Dense(num_classes, activation="relu")
        self.fc_upper = keras.layers.Dense(num_classes, activation="relu")
        self.fc_lower = keras.layers.Dense(num_classes, activation="relu")
        self.fc_out = keras.layers.Dense(num_classes, activation="softmax")

    def call(self, inputs):
        out = self.conv_1(inputs)
        out = self.init_bn(out)
        out = tf.nn.relu(out)
        out = self.pool_2(out)
        for res_block in [self.res_1_1, self.res_1_2, self.res_2_1, self.res_2_2, self.res_3_1, self.res_3_2]:
            out = res_block(out)
        
        num_out = self.res_num_1(out)
        num_out = self.res_num_2(num_out)
        num_out = self.num_avg_pool(num_out)
        num_out = self.num_flat(num_out)
        num_out = self.fc_num(num_out)

        upper_out = self.res_upper_1(out)
        upper_out = self.res_upper_2(upper_out)
        upper_out = self.res_upper_3(upper_out)
        upper_out = self.upper_avg_pool(upper_out)
        upper_out = self.upper_flat(upper_out)
        upper_out = self.fc_upper(upper_out)

        lower_out = self.res_lower_1(out)
        lower_out = self.res_lower_2(lower_out)
        lower_out = self.res_lower_3(lower_out)
        lower_out = self.lower_avg_pool(lower_out)
        lower_out = self.lower_flat(lower_out)
        lower_out = self.fc_lower(lower_out)

        out = keras.layers.Concatenate()([num_out, upper_out, lower_out])

        out = self.fc_out(out)
        return out



In [52]:
class Res_5x5_v6(keras.models.Model):

    def __init__(self, num_classes, **kwargs):
        """
            num_classes: number of classes in specific classification task.
        """
        super().__init__(**kwargs)
        input_ch = 64
        self.conv_1 = keras.layers.Conv2D(input_ch, (5, 5), strides=2,
                             padding="same", kernel_initializer="he_normal")
        self.init_bn = keras.layers.BatchNormalization()
        self.pool_2 = keras.layers.MaxPool2D(pool_size=(2, 2), strides=2, padding="same")
        self.res_1_1 = ResnetBlock(input_ch)
        self.res_1_2 = ResnetBlock(input_ch)
        self.res_2_1 = ResnetBlock(input_ch*2, down_sample=True)
        self.res_2_2 = ResnetBlock(input_ch*2)
        
        self.res_num_1 = ResnetBlock(input_ch*4, down_sample=True)
        self.res_num_2 = ResnetBlock(input_ch*4)
        self.res_num_3 = ResnetBlock(input_ch*8, down_sample=True)
        self.res_num_4 = ResnetBlock(input_ch*8)
        
        self.res_upper_1 = ResnetBlock(input_ch*4, down_sample=True)
        self.res_upper_2 = ResnetBlock(input_ch*4)
        self.res_upper_3 = ResnetBlock(input_ch*8, down_sample=True)
        self.res_upper_4 = ResnetBlock(input_ch*8)        
        self.res_upper_5 = ResnetBlock(input_ch*8)        

        self.res_lower_1 = ResnetBlock(input_ch*4, down_sample=True)
        self.res_lower_2 = ResnetBlock(input_ch*4)
        self.res_lower_3 = ResnetBlock(input_ch*8, down_sample=True)
        self.res_lower_4 = ResnetBlock(input_ch*8)
        self.res_lower_5 = ResnetBlock(input_ch*8)

        self.num_avg_pool = keras.layers.GlobalAveragePooling2D()
        self.upper_avg_pool = keras.layers.GlobalAveragePooling2D()
        self.lower_avg_pool = keras.layers.GlobalAveragePooling2D()

        self.num_flat = keras.layers.Flatten()
        self.upper_flat = keras.layers.Flatten()
        self.lower_flat = keras.layers.Flatten()

        self.fc_out = keras.layers.Dense(num_classes, activation="softmax")

    def call(self, inputs):
        out = self.conv_1(inputs)
        out = self.init_bn(out)
        out = tf.nn.relu(out)
        out = self.pool_2(out)
        for res_block in [self.res_1_1, self.res_1_2, self.res_2_1, self.res_2_2]:
            out = res_block(out)
        
        num_out = self.res_num_1(out)
        num_out = self.res_num_2(num_out)
        num_out = self.res_num_3(num_out)
        num_out = self.res_num_4(num_out)
        num_out = self.num_avg_pool(num_out)
        num_out = self.num_flat(num_out)

        upper_out = self.res_upper_1(out)
        upper_out = self.res_upper_2(upper_out)
        upper_out = self.res_upper_3(upper_out)
        upper_out = self.res_upper_4(upper_out)
        upper_out = self.res_upper_5(upper_out)
        upper_out = self.upper_avg_pool(upper_out)
        upper_out = self.upper_flat(upper_out)

        lower_out = self.res_lower_1(out)
        lower_out = self.res_lower_2(lower_out)
        lower_out = self.res_lower_3(lower_out)
        lower_out = self.res_lower_4(lower_out)
        lower_out = self.res_lower_5(lower_out)
        lower_out = self.lower_avg_pool(lower_out)
        lower_out = self.lower_flat(lower_out)

        out = keras.layers.Concatenate()([num_out, upper_out, lower_out])

        out = self.fc_out(out)
        return out



In [53]:
resnet_model = Res_5x5_v6(62)
resnet_model.build(input_shape = (None,28,28,1))
#use categorical_crossentropy since the label is one-hot encoded

# from keras.optimizers import SGD
# opt = SGD(learning_rate=0.1,momentum=0.9,decay = 1e-04) #parameters suggested by He [1]
resnet_model.compile(optimizer = "adam",loss='categorical_crossentropy', metrics=["accuracy"]) 

In [54]:
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)

es = EarlyStopping(patience= 5, restore_best_weights=True, monitor="val_accuracy")

batch_size = 256
#I did not use cross validation, so the validate performance is not accurate.
STEPS = len(X_train) / 256

history = resnet_model.fit(aug.flow(X_train,y_train,batch_size = batch_size), 
      steps_per_epoch=STEPS, 
      batch_size = batch_size, 
      epochs=50, 
      validation_data=(X_val, y_val),
      callbacks=[es])

Epoch 1/50
2181/2181 [==============================] - 106s 47ms/step - loss: 0.8764 - accuracy: 0.7224 - val_loss: 0.6341 - val_accuracy: 0.7901
Epoch 2/50
2181/2181 [==============================] - 102s 47ms/step - loss: 0.6064 - accuracy: 0.7930 - val_loss: 0.6100 - val_accuracy: 0.7914
Epoch 3/50
2181/2181 [==============================] - 104s 47ms/step - loss: 0.5639 - accuracy: 0.8062 - val_loss: 0.5303 - val_accuracy: 0.8076
Epoch 4/50
2181/2181 [==============================] - 104s 48ms/step - loss: 0.5376 - accuracy: 0.8140 - val_loss: 0.6508 - val_accuracy: 0.8314
Epoch 5/50
2181/2181 [==============================] - 104s 48ms/step - loss: 0.5195 - accuracy: 0.8191 - val_loss: 0.4793 - val_accuracy: 0.8309
Epoch 6/50
2181/2181 [==============================] - 104s 48ms/step - loss: 0.4961 - accuracy: 0.8252 - val_loss: 0.5075 - val_accuracy: 0.8106
Epoch 7/50
2181/2181 [==============================] - 103s 47ms/step - loss: 0.4863 - accuracy: 0.8275 - val_loss: 0

In [55]:
weight_path = '../model/ResBaseWeight_5x5_v6/ResBaseWeight_5x5_v6.ckpt'
resnet_model.save_weights(weight_path)

In [56]:
resnet_model.evaluate(X_val,y_val)

4363/4363 [==============================] - 47s 11ms/step - loss: 0.4483 - accuracy: 0.8411


[0.4483068883419037, 0.8410883545875549]

In [57]:
resnet_model.evaluate(X_test,y_test)

3636/3636 [==============================] - 39s 11ms/step - loss: 0.4468 - accuracy: 0.8410


[0.44684842228889465, 0.840985894203186]

Res50 model

In [ ]:
from keras.callbacks import EarlyStopping
from keras.preprocessing.image import ImageDataGenerator

aug = ImageDataGenerator(
    rotation_range=30,
    width_shift_range=0.1,
    height_shift_range=0.1,
    shear_range=0.2,
    zoom_range=0.2,
    horizontal_flip=True,
    fill_mode="nearest",
)

es = EarlyStopping(patience= 8, restore_best_weights=True)
model_checkpoint_cb = keras.callbacks.ModelCheckpoint("new_model.h5", save_best_only=True)

batch_size = 256
#I did not use cross validation, so the validate performance is not accurate.
STEPS = len(X_train) / 256

history = resnet_model.fit(aug.flow(X_train,y_train,batch_size = batch_size), 
      steps_per_epoch=STEPS, 
      batch_size = batch_size, 
      epochs=50, 
      validation_data=(X_train, y_train),
      callbacks=[es])

In [ ]:
input_layer = tf.keras.layers.Resizing(height = 32, width = 32, input_shape = [28, 28, 3])

input = tf.keras.Input(shape=(32,32,3))
efnet = tf.keras.applications.ResNet50(weights='imagenet',
                                             include_top = False, 
                                             input_tensor = input)
# Now that we apply global max pooling.
gap = tf.keras.layers.GlobalMaxPooling2D()

# Finally, we add a classification layer.
output = tf.keras.layers.Dense(62, activation='softmax', use_bias=True)

# bind all
func_model = keras.models.Sequential()
func_model.add(input_layer)
func_model.add(efnet)
func_model.add(gap)
func_model.add(output)

In [ ]:
func_model.summary()

In [ ]:
from keras.callbacks import EarlyStopping

res50_checkpoint = keras.callbacks.ModelCheckpoint("res50.h5", save_best_only=True)
es = EarlyStopping(patience= 8, restore_best_weights=True)

# func_model.compile(
#           loss  = 'sparse_categorical_crossentropy',
#           metrics = tf.keras.metrics.SparseCategoricalAccuracy(),
#           optimizer = tf.keras.optimizers.Adam())

func_model.compile(optimizer='adam',
          loss='sparse_categorical_crossentropy', 
          metrics=['accuracy'])


In [ ]:
func_model.fit(X_train, y_train, epochs=5, verbose = 2,
          callbacks=[es, res50_checkpoint])